In [2]:
# from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
import math
import random

In [6]:
# female_stereotype_list
female_stereotype = open("female_stereotype.txt", "r") 
data = female_stereotype.read()
female_stereotype_list = data.split("\n")
female_stereotype.close()


# male_stereotype_liste
male_stereotype = open("male_stereotype.txt", "r")
data = male_stereotype.read()
male_stereotype_list = data.split("\n")
male_stereotype.close()



In [10]:
# # model1 Pretrained model
from transformers import BertTokenizer, BertModel
# import torch
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# model1
import torch
# # Load BertForSequenceClassification, the pretrained BERT model with a single 
# # linear classification layer on top. 
model1 = BertModel.from_pretrained('bert-large-uncased')

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:



# model2 Context-debias
import torch
# tokenizer2 = BertTokenizer.from_pretrained('bert-base-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model2 = BertModel.from_pretrained('bert-large-uncased', state_dict=torch.load("../../../../kaneko_debiased_bert/context-debias-main/debiased_models/42/bert_large/checkpoint-best/pytorch_model.bin"))


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:


# model3 Sent-debias
import torch
# tokenizer2 = BertTokenizer.from_pretrained('bert-base-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model3 = BertModel.from_pretrained("bert-large-uncased")



Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:

# model4 CDA
import torch
# tokenizer2 = BertTokenizer.from_pretrained('bert-base-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model4 = BertModel.from_pretrained("bert-large-uncased", state_dict=torch.load("../../../../CDA/counterfactual-data-substitution-master/zari-bert-cda/pytorch_model.bin"))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
models = [model1, model2, model3, model4]
tokenizers = [tokenizer]

In [16]:
female_list = list(female_stereotype_list)
male_list = list(male_stereotype_list)

In [17]:
# # gendered_word_list
# gendered_word_list = [["she", "he"],["female", "male"], ["woman", "man"], ["girl", "boy"], ["sister", "brother"], ["daughter", "son"], ["mother", "father"]]
XX = ["female", "woman", "girl", "sister", "daughter", "mother", "aunt", "grandmother"]
YY = ["male", "man", "boy", "brother", "son", "father", "uncle", "grandfather"]

In [18]:
templates = ["this is the _.", "that is the _.", "there is the _.", "here is the _.", "the _ is here.", "the _ is there.", "it is the _.","the _ is a [MASK]."]

In [19]:
temps = ["temp1", "temp2", "temp3", "temp4","temp5", "temp6", "temp7", "temp8", "temp9", "temp10", "temp11"]

In [20]:
def get_index(sentence, word):
    toks = tokenizer(sentence).input_ids
    wordpieces = tokenizer(word).input_ids
#     print(toks)
    word = wordpieces[1] # use first wordpiece
    for i, t in enumerate(toks):
        if t == word:
            return i

In [34]:
# debias embeddings

import pickle
import logging
from sklearn.decomposition import PCA

logger = logging.getLogger(__name__)

def load_gender_pair_embeddings():
    with open('../../../../liang_sent_debias-master/debias-BERT/experiments/4_bert_large_saved_embs/num7993_a_pretrained.pkl', 'rb') as f:
        all_embeddings_a = pickle.load(f)

    with open('../../../../liang_sent_debias-master/debias-BERT/experiments/4_bert_large_saved_embs/num7993_b_pretrained.pkl', 'rb') as f:
        all_embeddings_b = pickle.load(f)


    means = (all_embeddings_a + all_embeddings_b) / 2.0
    all_embeddings_a -= means
    all_embeddings_b -= means
    all_embeddings = np.concatenate([all_embeddings_a, all_embeddings_b], axis=0)
    return all_embeddings





all_embeddings = load_gender_pair_embeddings()


def doPCA(matrix, num_components=10):
	pca = PCA(n_components=num_components, svd_solver="auto")
	pca.fit(matrix) # Produce different results each time...
	return pca


def drop_bias(u, v):
    return u - torch.ger(torch.matmul(u, v), v) / v.dot(v)


def get_gender_dir(k):
    gender_dir = doPCA(all_embeddings).components_[:k]
    # if (not keepdims):
    gender_dir = np.mean(gender_dir, axis=0)
    logger.info("gender direction={} {} {}".format(gender_dir.shape,
            type(gender_dir), gender_dir[:10]))
    gender_dir = torch.from_numpy(gender_dir)
    return gender_dir

In [35]:

# SWP first embedding
def sentence_embedding(template, word, model_ind):
    sentence = template.replace("_", word)
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = models[model_ind](**inputs)
    last_hidden_states = outputs.last_hidden_state
    token_embeddings = last_hidden_states
    input_mask_expanded = inputs.attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    start = get_index(sentence, word)
    embeddings = token_embeddings[0][start]
    return embeddings.cpu().detach().numpy()


# SWP first embedding
def model3_sentence_embedding(template, word, model_ind):
    model = models[model_ind]
    sentence = template.replace("_", word)
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    token_embeddings = last_hidden_states
    for t in range(token_embeddings.shape[1]):
        token_embeddings[:, t] = drop_bias(token_embeddings[:, t], gender_dir)
        token_embeddings = torch.nn.functional.normalize(token_embeddings, p=2, dim=-1)
    input_mask_expanded = inputs.attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    start = get_index(sentence, word)
    embeddings = token_embeddings[0][start]
    return embeddings.cpu().detach().numpy()

In [36]:
def cossim(x, y):
    return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))

In [37]:
def construct_cossim_lookup(XY, AB):
    """
    XY: mapping from target string to target vector (either in X or Y)
    AB: mapping from attribute string to attribute vectore (either in A or B)
    Returns an array of size (len(XY), len(AB)) containing cosine similarities
    between items in XY and items in AB.
    """

    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
            cossims[xy, ab] = cossim(XY[xy], AB[ab])
    return cossims


In [38]:
def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [39]:

def s_XAB(A, s_wAB_memo):
    return s_wAB_memo[A].sum()

In [40]:
def s_wAB(X, Y, cossims):
    """
    Return vector of s(w, A, B) across w, where
        s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
    """
    return cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)

In [41]:
def s_XAB_df(A, B, s_wAB_memo):
    df1 = pd.DataFrame(s_wAB_memo[A])
    df2 = pd.DataFrame(s_wAB_memo[B])
    return df1, df2

In [42]:
def s_XYAB(A, B, s_wAB_memo):
    r"""
    Given indices of target concept X and precomputed s_wAB values,
    the WEAT test statistic for p-value computation.
    """
    return s_XAB(A, s_wAB_memo) - s_XAB(B, s_wAB_memo)

In [43]:
def WEAT_test(X, Y, A, B, n_samples, cossims):
    ''' Compute the p-val for the permutation test, which is defined as
        the probability that a random even partition X_i, Y_i of X u Y
        satisfies P[s(X_i, Y_i, A, B) > s(X, Y, A, B)]
    '''
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    A = np.array(list(A), dtype=np.int)
    B = np.array(list(B), dtype=np.int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(X, Y, cossims=cossims)
    XY = np.concatenate((X, Y))


    s = s_XYAB(A, B, s_wAB_memo)
    return s


def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [44]:
def get_effect_size(df1, df2, k=0):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [45]:
gender_dir = get_gender_dir(1)

template_score_dict = {}
for ind, template in enumerate(templates):
    score_dict = {}
    attribute_template = template
    target_template = template
    for model_ind in range(len(models)):
        print(model_ind)
        if model_ind == 2:
            X = {"x" + str(j): sentence_embedding(attribute_template, j, model_ind) for j in XX}
            Y = {"y" + str(j): sentence_embedding(attribute_template, j, model_ind) for j in YY}
            (X, Y) = convert_keys_to_ints(X, Y)
            XY = X.copy()
            XY.update(Y)
            X = np.array(list(X), dtype=np.int)
            Y = np.array(list(Y), dtype=np.int)
            AA = female_list
            BB = male_list

            A = {"a" + str(j): model3_sentence_embedding(target_template, j, model_ind) for j in AA}
            B = {"b" + str(j): model3_sentence_embedding(target_template, j, model_ind) for j in BB}

            (A, B) = convert_keys_to_ints(A, B)


            AB = A.copy()
            AB.update(B)

            cossims = construct_cossim_lookup(XY, AB)
            A = np.array(list(A), dtype=np.int)
            B = np.array(list(B), dtype=np.int)


            s_wAB_memo = s_wAB(X, Y, cossims=cossims)
            df1,df2 = s_XAB_df(A, B, s_wAB_memo)
            effect_size = get_effect_size(df1, df2)
            score_dict[model_ind] = effect_size
            print(score_dict[model_ind])
        else:
            X = {"x" + str(j): sentence_embedding(attribute_template, j, model_ind) for j in XX}
            Y = {"y" + str(j): sentence_embedding(attribute_template, j, model_ind) for j in YY}
            (X, Y) = convert_keys_to_ints(X, Y)
            XY = X.copy()
            XY.update(Y)
            X = np.array(list(X), dtype=np.int)
            Y = np.array(list(Y), dtype=np.int)
            AA = female_list
            BB = male_list

            A = {"a" + str(j): sentence_embedding(target_template, j, model_ind) for j in AA}
            B = {"b" + str(j): sentence_embedding(target_template, j, model_ind) for j in BB}

            (A, B) = convert_keys_to_ints(A, B)


            AB = A.copy()
            AB.update(B)

            cossims = construct_cossim_lookup(XY, AB)
            A = np.array(list(A), dtype=np.int)
            B = np.array(list(B), dtype=np.int)


            s_wAB_memo = s_wAB(X, Y, cossims=cossims)
            df1,df2 = s_XAB_df(A, B, s_wAB_memo)
            effect_size = get_effect_size(df1, df2)
            score_dict[model_ind] = effect_size
            print(score_dict[model_ind])
    template_score_dict[temps[ind]] = score_dict


0
1.6239635580176857
1
0.3652346837031964
2
1.6242898579344123
3
1.340075283883311
0
1.6071278176986008
1
0.31627028386638095
2
1.608358418028016
3
1.5089758469212187
0
1.5217741410325767
1
0.05824415570862318
2
1.5218751527597913
3
1.3201387818889876
0
1.4770595779850684
1
0.5257678541030162
2
1.4775274962839435
3
1.3914654641989752
0
1.570224662780473
1
0.3905907087468089
2
1.570996816935041
3
0.4497110054647657
0
1.6475621836149925
1
-0.05282674089465071
2
1.6479510380072386
3
0.4897858485806677
0
1.532952782910619
1
0.3105679679696524
2
1.5334169033812621
3
1.2228405926748434
0
1.6177424708230268
1
-0.147007102125903
2
1.6172852952823416
3
0.7817368119372631


In [48]:
score_df = pd.DataFrame(template_score_dict)

In [49]:
score_df

,temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8
0,1.623964,1.607128,1.521774,1.477060,1.570225,1.647562,1.532953,1.617742
1,0.365235,0.316270,0.058244,0.525768,0.390591,-0.052827,0.310568,-0.147007
2,1.624290,1.608358,1.521875,1.477527,1.570997,1.647951,1.533417,1.617285
3,1.340075,1.508976,1.320139,1.391465,0.449711,0.489786,1.222841,0.781737
